In [1]:
import pandas as pd 
import numpy as np 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Cargo mi set de datos
df = pd.read_csv('/content/drive/MyDrive/Inteligencia Artificial/segundo año/cuarto/AA2/parcial/AA2-IE1-2022-10-04/hongos-IE1.csv')
df.head()

,cap-shape,cap-surface,cap-color,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-surface-above-ring,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,target
0,5.0,0.0,4.0,5.0,1.0,0.0,0.0,7.0,1.0,2.0,...,6.0,6.0,0.0,2.0,1.0,4.0,2.0,5.0,0.0,e
1,2.0,0.0,3.0,2.0,1.0,0.0,0.0,3.0,0.0,1.0,...,6.0,6.0,0.0,2.0,1.0,2.0,1.0,4.0,0.0,p
2,0.0,0.0,3.0,5.0,1.0,1.0,0.0,2.0,0.0,2.0,...,7.0,7.0,0.0,2.0,2.0,4.0,7.0,2.0,1.0,e
3,2.0,3.0,1.0,4.0,0.0,0.0,0.0,10.0,0.0,1.0,...,1.0,1.0,0.0,2.0,0.0,3.0,7.0,1.0,0.0,p
4,2.0,0.0,8.0,5.0,1.0,1.0,0.0,7.0,1.0,2.0,...,7.0,7.0,0.0,2.0,1.0,0.0,2.0,3.0,1.0,e


In [11]:
# Establezco que mi target sea de tipo numerico, siendo 0=e y 1=p
df['target']=df['target'].replace('e',0)
df['target']=df['target'].replace('p',1)

In [20]:
# Separo mis variables x e y
x= df.drop('target',axis=1)
y= df['target']

In [21]:
# Genero mis separaciones entre train y test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=123)

In [41]:
# Entreno mi modelo SVC con CV
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
modeloSVC= SVC()
parametros = {'kernel':['rbf'], 'C':[1e-6,1e-5,1e-4,0.0001,0.001,0.01,0.1,1,10],'gamma':['scale','auto']}
GridSVC= GridSearchCV(modeloSVC,param_grid=parametros,scoring=['accuracy','f1'],cv=5,refit='accuracy')
GridSVC.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1e-06, 1e-05, 0.0001, 0.0001, 0.001, 0.01, 0.1,
                               1, 10],
                         'gamma': ['scale', 'auto'], 'kernel': ['rbf']},
             refit='accuracy', scoring=['accuracy', 'f1'])

In [51]:
# Guardo mis mejores hiperparametros 
parametros=GridSVC.best_params_

In [65]:
#Genero listas con informacion que me interesa de cv_results_ para luego analizarla
accuracy_total=GridSVC.cv_results_['mean_test_accuracy']
f1_total=GridSVC.cv_results_['mean_test_f1']
desv_accuracy_total = GridSVC.cv_results_['std_test_accuracy']
desv_f1_total=GridSVC.cv_results_['mean_test_f1']

In [64]:
# Para encontrar mi valor de accuracy multiplicando 3 veces su desviacion estandar lo buscaré por indices 
acuracy=accuracy_total[GridSVC.best_index_]
f1=f1_total[GridSVC.best_index_]
desv_accuracy=desv_accuracy_total[GridSVC.best_index_]
desv_f1= desv_f1_total[GridSVC.best_index_]
print('----------------------------------------------')
print('AC = ',acuracy,'+/-',3*desv_accuracy,'( 99% )')
print('----------------------------------------------')
print('F1 = ',f1,'+/-',3*desv_f1,'( 99% )')

----------------------------------------------
AC =  0.9984610643593694 +/- 0.005055901331052414 ( 99% )
----------------------------------------------
F1 =  0.9983989727126806 +/- 2.995196918138042 ( 99% )


In [74]:
# Por ultimo, una vez encontrado mis mejores hiperparametros, GridSearchCV reconoce al mejor y lo utiliza para calcular el score con 
# los datos de x_test e y_test
modelo_final_SVC=GridSVC.score(x_test,y_test)
modelo_final_SVC

1.0

In [71]:
# guardo el modelo con los mejores parametros, para ser usado como modelo de produccion, a diferencia de que ahora lo entreno 
# con todos los valores de x e y
modelo_produccion= GridSVC.best_estimator_
modelo_produccion.fit(x,y)

SVC(C=10, gamma='auto')

In [73]:
# Guardo mi modelo generado 
import pickle
archivo = 'modelo.sav '
pickle.dump(modelo_produccion, open(archivo, 'wb'))